<a href="https://colab.research.google.com/github/Fambly/PRiR_lab_8/blob/main/Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [4]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-4_l_f1ra
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-4_l_f1ra
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=8b785deb93d13b231d839bd2f3dda3403c28323664da6cb82835a74e9ebad9c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-18sjvg4a/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


Aby była możliwość skompilowania programów poniżej należy w oknie Google Colab w zakładce "Środowisko wykonawcze" zamienić "none" na "GPU" a następnie wykonać 
powyższe 3 linijki kodu by zainstalować środowisko

***CPU***:

In [19]:
%%cu 
#include<stdio.h>

#define DIAMETER 1000

struct cuComplex
{
  float r,i;
  float magnitude(void)
  {
      return r*r+i*i;
  }
 
  cuComplex(float a, float b): r(a), i(b){}
  cuComplex operator*(const cuComplex &a)
  {
       return cuComplex(r*a.r-i*a.i,i*a.r+r*a.i);
  }
  cuComplex operator+(const cuComplex &a)
  {
       return cuComplex(r+a.r,i+a.i); 
  }
};


int fraktal(int x, int y)
{
  float Half_ofDiameter = DIAMETER/2;
  const float scale=1.5;
  float jx = scale * (float)(Half_ofDiameter-x) / Half_ofDiameter;
  float jy = scale * (float)(Half_ofDiameter-y) / Half_ofDiameter;
  cuComplex c(jx, jy);
  cuComplex a(0, 0);
  for(int i = 0; i < 200; i++)
  {
    a = a * a + c;
    if(a.magnitude() > 1000) return 0;
  }
  return 1;
}


void kernel(unsigned char *data)
{
  for(int y = 0; y < DIAMETER; y++)
  {
    for(int x = 0; x < DIAMETER; x++)
    {
      int val_of_0 = (x+y*DIAMETER)*4+0;
      int val_of_1 = val_of_0 + 1;
      int val_of_2 = val_of_0 + 2;
      int val_of_3 = val_of_0 + 3;
      int Value = fraktal(x,y);
      data[val_of_0] = 255 * Value;
      data[val_of_1] = 0;
      data[val_of_2] = 0;
      data[val_of_3] = 255;
    }
  }
}                                                                                                                                                                                                    

struct DataBlock
{
  unsigned char *dev_bitmap;
};

void output(unsigned char *data)
{
  FILE *fp = fopen("CPU.ppm","w");
  fprintf(fp, "P3\n%d %d\n255\n", DIAMETER, DIAMETER);
  for(int y = 0; y < DIAMETER; y++)
  {
    for(int x = 0;x < DIAMETER; x++)
    {
      int val_of_0 = (x + y * DIAMETER) * 4 + 0;
      int val_of_1 = val_of_0 + 1;
      int val_of_2 = val_of_0 + 2;
      fprintf(fp,"\n%d %d %d", data[val_of_1], data[val_of_0], data[val_of_2]);
    }
  }
  fclose(fp);
}


int main(void)
{
    
  clock_t start = clock();
  clock_t end;
  double time_taken;

  DataBlock data;
  unsigned char *bitmap = (unsigned char*)malloc(DIAMETER*DIAMETER*4*sizeof(unsigned char));
  kernel(bitmap);
  output(bitmap);
  
  end = clock();
  time_taken = ((double)(end - start))/CLOCKS_PER_SEC;
  printf("Time the code took to end: %f seconds", time_taken);
}

Time the code took to end: 1.604463 seconds


Aby wyświetlić efekt programu należy po lewej stronie okna google colab kliknąć na ikonę foldera następnie na pliku "CPU.ppm" w menu pod trzema kropkami kliknąć "Pobierz".
Pobrany plik w formacie .ppm można przekonwertować w format .jpg / .png itp za pomocą dostępnych converterów Online by zobaczyć obraz 
Drugim sposobem jest otworzenie pliku za pomocą programu o nazwie IrfanVieww


***GPU***:

In [21]:
%%cu
#include<stdio.h>

#define DIAMETER 1000


struct cuComplex
{
  float r,i;
  __device__ cuComplex(float a, float b): r(a), i(b){}
  __device__ float magnitude2(void)
  { 
      return r * r + i * i;
  }
  __device__ cuComplex operator*(const cuComplex &a)
  {
       return cuComplex(r*a.r-i*a.i,i*a.r+r*a.i);
  }
  __device__ cuComplex operator+(const cuComplex &a)
  {
       return cuComplex( r + a.r, i + a.i);
  }
};

__device__  int mandelbrot(int x, int y)
{
  float Half_ofDiameter = DIAMETER / 2;
  const float scale=1.5;
  float jx = scale*(float)(Half_ofDiameter-x) / Half_ofDiameter;
  float jy = scale*(float)(Half_ofDiameter-y) / Half_ofDiameter;
  cuComplex c(jx, jy);
  cuComplex a(0, 0);
  for(int i = 0; i < 200; i++)
  {
    a= a * a + c;
    if(a.magnitude2() > 1000)
    {
        return 0;
    }  
  }
  return 1;
}

__global__ void kernel(unsigned char *data)
{
  int x = blockIdx.x;
  int y = blockIdx.y;
  int val_of_0 = (x+y*gridDim.x)*4+0;
  int val_of_1 = val_of_0 + 1;
  int val_of_2 = val_of_0 + 2;
  int val_of_3 = val_of_0 + 3;
  int Value=mandelbrot(x,y);
  data[val_of_0]=255*Value;
  data[val_of_1]=0;
  data[val_of_2]=0;
  data[val_of_3]=255;
}

struct DataBlock{
  unsigned char *bitmap_Data;
};

void output(unsigned char *data){
  FILE *fp = fopen("GPU.ppm","w");
  fprintf(fp, "P3\n%d %d\n255\n", DIAMETER, DIAMETER);
  for(int y = 0; y < DIAMETER; y++)
  {
    for(int x = 0; x < DIAMETER; x++)
    {
      int val_of_0 = (x + y * DIAMETER) * 4 + 0;
      int val_of_1 = val_of_0 + 1;
      int val_of_2 = val_of_0 + 2;
      fprintf(fp,"\n%d %d %d", data[val_of_1], data[val_of_0],data[val_of_2]);
    }
  }
  fclose(fp);
}                                                                                                                                                                                                              

int main(void)
{
  clock_t start = clock();
  clock_t end;
  double time_taken;

  DataBlock data;
  unsigned char *bitmap = (unsigned char*)malloc(DIAMETER*DIAMETER*4*sizeof(unsigned char));
  int size = DIAMETER*DIAMETER*4;
  unsigned char *bitmap_Data;
  cudaMalloc((void**)&bitmap_Data, size);
  data.bitmap_Data = bitmap_Data;
  dim3 grid(DIAMETER, DIAMETER);
  kernel<<<grid,1>>>(bitmap_Data);
  cudaMemcpy(bitmap, bitmap_Data, size, cudaMemcpyDeviceToHost);
  cudaFree(bitmap_Data);
  output(bitmap);

  end = clock();
  time_taken = ((double)(end - start))/CLOCKS_PER_SEC;
  printf("Time the code took to end: %f seconds", time_taken);
}

Time the code took to end: 0.875244 seconds


Aby wyświetlić efekt programu należy po lewej stronie okna google colab kliknąć na ikonę foldera następnie na pliku "GPU.ppm" w menu pod trzema kropkami kliknąć "Pobierz".
Pobrany plik w formacie .ppm można przekonwertować w format .jpg / .png itp za pomocą dostępnych converterów Online by zobaczyć obraz 
Drugim sposobem jest otworzenie pliku za pomocą programu o nazwie IrfanVieww

Średni zysk w czasie obliczeń między CPU a GPU wynosi 50% na danym komputerze
(1.6s => 0.8s)